In [11]:
#!pip install mlrun==0.6.0-rc7


In [39]:
from mlrun.run import new_function
 
sj = new_function(kind='spark', command='/User/parquez/functions/kv_to_parquet.py', name='test-spark') # /User isn't supported at this stage
 
sj.with_driver_limits(cpu="1300m")
sj.with_driver_requests(cpu=1, mem="512m") # gpu_type & gpus=<number_of_gpus> are supported too
sj.with_executor_limits(cpu="1400m")
sj.with_executor_requests(cpu=1, mem="512m") # gpu_type & gpus=<number_of_gpus> are supported too
 
sj.with_igz_spark() # Adds fuse, daemon & iguazio's jars support

# Args are also supported:
# sj.spec.args = ['-arg1', '-arg2']
 
sj.spec.replicas = 2 # Number of executors
 
sj.deploy() # Rebuilds the image with MLRun - This is needed in order to support artifact logging etc. This step is too long (~3 minutes)
 
sr = sj.run(artifact_path="/User/artifacts")

> 2020-12-22 12:47:51,225 [info] running build to add mlrun package, set with_mlrun=False to skip if its already in the image
> 2020-12-22 12:47:51,235 [info] starting remote build, image: .mlrun/func-default-test-spark-latest
E1222 12:47:54.466628       1 aws_credentials.go:77] while getting AWS credentials NoCredentialProviders: no valid providers in chain. Deprecated.
	For verbose messaging see aws.Config.CredentialsChainVerboseErrors
INFO[0000] Retrieving image manifest datanode-registry.iguazio-platform.app.app-lab-azur-3.iguazio-cd0.com:80/iguazio/spark-app:3.0_b5914_20201219122232 
INFO[0000] Retrieving image manifest datanode-registry.iguazio-platform.app.app-lab-azur-3.iguazio-cd0.com:80/iguazio/spark-app:3.0_b5914_20201219122232 
INFO[0000] Built cross stage deps: map[]                
INFO[0000] Retrieving image manifest datanode-registry.iguazio-platform.app.app-lab-azur-3.iguazio-cd0.com:80/iguazio/spark-app:3.0_b5914_20201219122232 
INFO[0000] Retrieving image manifest da

project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
default,...18c8c317,0,Dec 22 12:51:03,error,test-spark,v3io_user=aviakind=sparkowner=avia,,,,


to track results use .show() or .logs() or in CLI: 
!mlrun get run 8d76b482fbc74a7ea46dc4ce18c8c317 --project default , !mlrun logs 8d76b482fbc74a7ea46dc4ce18c8c317 --project default
> 2020-12-22 12:53:56,701 [info] run executed, status=error
runtime error: None


RunError: None

In [32]:
import os.path
from os import path

def generate_kv_parquet_path(container='parquez',
                             table='faker',
                             compress_type='parquet',
                             partition_by='h',
                             real_time_window='0h'):
    real_time_window_delta = int(real_time_window[:-1])
    print(real_time_window_delta)
    from datetime import datetime, timezone, timedelta
    from dateutil.relativedelta import relativedelta
    current_date_path = None
    if partition_by == 'h':
        current_date_path = (datetime.now(timezone.utc) - timedelta(hours=real_time_window_delta)).strftime(
            "year=%Y/month=%m/day=%d/hour=%H")
    elif partition_by == 'd':
        current_date_path = (datetime.now(timezone.utc) - timedelta(days=real_time_window_delta)).strftime(
            "year=%Y/month=%m/day=%d")
    elif partition_by == 'm':
        current_date_path = (datetime.now(timezone.utc) - relativedelta(months=real_time_window_delta)).strftime(
            "year=%Y/month=%m")
    elif partition_by == 'y':
        current_date_path = (datetime.now(timezone.utc) - relativedelta(years=real_time_window_delta)).strftime(
            "year=%Y")
    kv_path = "v3io://{}/{}/{}/".format(container, table, current_date_path)
    parquet_path = "v3io://{}/{}_{}/{}/".format(container, table, compress_type, current_date_path)
    fuse_kv_path =  "/v3io/{}/{}/{}/".format(container, table, current_date_path)
    print("kv path: {} , parquet_path : {} :fuse_kv_path {}".format(kv_path, parquet_path, fuse_kv_path))
    return {'kv_path': kv_path, 'parquet_path': parquet_path, 'fuse_kv_path': fuse_kv_path}

paths = generate_kv_parquet_path()
print(path)

0
kv path: v3io://parquez/faker/year=2020/month=12/day=22/hour=11/ , parquet_path : v3io://parquez/faker_parquet/year=2020/month=12/day=22/hour=11/ :fuse_kv_path /v3io/parquez/faker/year=2020/month=12/day=22/hour=11/
<module 'posixpath' from '/conda/lib/python3.7/posixpath.py'>


In [33]:
if path.isdir(paths['fuse_kv_path']):  
    print('wow')
else:
    print("Directory {} Doesnt exist".format(paths['fuse_kv_path']))    

wow


In [22]:
if path.isdir('/v3io/parquez/faker/year=2020/'):  
    print('wow')

wow


In [37]:
from pyspark.sql import SparkSession
# METHOD I:
#   Update Spark configurations of the following two extraClassPath with the JDBC driver location
#   prior to initiating a Spark session:
#      spark.driver.extraClassPath
#      spark.executor.extraClassPath
#
# NOTE:
# If you don't connnect to mysql, replace the mysql's connector by the other database's JDBC connector 
# in the following two extraClassPath.
#
# Initiate a Spark Session
spark = SparkSession.builder.appName("Spark JDBC to Databases - ipynb").getOrCreate()

df = spark.read.format('io.iguaz.v3io.spark.sql.kv').load('v3io://parquez/faker/year=2020/month=12/day=22/hour=11')
df.show()
#df.repartition(coalesce).write.mode('overwrite').parquet(paths['parquet_path'])

+--------------+------------------+--------------------+----+--------------------+--------------------+----+----------------+--------------------+----------+--------------------+----------------+----------------+--------------+---+-----+
|      JobTitle|              Name|             Company|year|               Email|            Password|hour|             key|        HackerPhrase|     Phone|            BeerName|              BS|CreditCardNumber|         Color|day|month|
+--------------+------------------+--------------------+----+--------------------+--------------------+----+----------------+--------------------+----------+--------------------+----------------+----------------+--------------+---+-----+
|Representative|  Vivienne Kreiger|Eichmann, Stroman...|2020|idellaschuppe@kih...|@z =3%OOTA ITC1Z+...|  11|ZMgOcJG0hde60u9t|Try to connect th...|8261744221|Trappistes Rochef...|           brand|2720990982779154|        Orchid| 22|   12|
|     Executive|        Coy Rippin|McLaughlin, B